<center><img src="http://www.nasa.gov/sites/all/themes/custom/nasatwo/images/nasa-logo.svg"></center>

<center>
<h1><font size="+3">GSFC Python Bootcamp</font></h1>
</center>

---

<center><h1> <font color="red">Introduction to Dask</font></h1></center>

## <font color="red">Reference Document</font>

- <a href="https://docs.dask.org/en/latest/why.html">Why Dask?</a>
- <a href="https://www.manning.com/books/data-science-with-python-and-dask">Data Science with Python and Dask</a>
- <a href="https://www.manifold.ai/dask-and-machine-learning-preprocessing-tutorial">Dask and Machine Learning: Preprocessing Tutorial</a>
- <a href="https://www.youtube.com/watch?v=uGy5gT2vLdI&feature=youtu.be"> Working with the Python DASK library (video)</a>

![fig_dask](https://miro.medium.com/max/1000/1*D6mSsdWECFLn6wJne4VTjg.png)


### What is Dask?

- A flexible library for parallel computing in Python that makes it easy to build intuitive workflows for ingesting and analyzing large, distributed datasets. 
- A native parallel analytics tool designed to integrate seamlessly with Numpy, Pandas, and Scikit-Learn. 
- An out-of-core (data is read into memory from disk on an as-needed basis) parallelization library that seamlessly integrates with existing NumPy and Pandas data structures to address the following:
     * The vailable dataset does not fit in memory of a single machine.
     * The data processing task is time consuming and needs to be sped up.
- Orchestrates parallel threads or processes for us and help speed up processing times.

Dask consists of several different components and APIs, which can be categorized into three layers: the scheduler, low-level APIs, and high-level APIs.

- Dask provides a few high-level constructs called Dask Bags, Dask DataFrames, and Dask Arrays. They provide an easy-to-use interface to parallelize many of the typical data transformations in ML workflows. 
- Dask allows the creation of highly customized job execution graphs by using their extensive Python API (e.g., `dask.delayed`) and integration with existing data structures.

![fig_layers](https://dpzbhybb2pdcj.cloudfront.net/daniel/HighResolutionFigures/figure_1-1.png)
Image Source: 


The diagram below describes the steps Dask takes to manipulate data.

![fig_proc](https://www.manifold.ai/hs-fs/hubfs/Blog%20Post%20Illos/ML%20pipelines%20-%20dask%20single%20machine.jpeg?width=600&name=ML%20pipelines%20-%20dask%20single%20machine.jpeg)
Image Source: www.manifold.ai


**Advantages of Using Dask**

- Fully implemented in Python and natively scales NumPy, Pandas, and scikit-learn.
- Can be used effectively to work with both medium datasets on a single machine and large datasets on a cluster.
- Can be used as a general framework for parallelizing most Python objects.
- Has a very low configuration and maintenance overhead.





In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask
import dask.array as da
import dask.dataframe as dd
from dask.diagnostics import ProgressBar 

## Dask Array

- Dask arrays coordinate many Numpy arrays, arranged into chunks within a grid. 
- They support a large subset of the Numpy API.

![fig_array](https://miro.medium.com/max/1388/1*JfQnXJ5_R104bPyE8_XhwQ.png)

**Create a Dask Array**

- Create a 20000x20000 array of random numbers, represented as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly). 
- There are 100 (10x10) numpy arrays of size 1000x1000.

In [ ]:
x = da.random.random((20000, 20000), chunks=(1000, 1000))
x

We can use Numpy syntax:

In [ ]:
y = x + x.T
y.shape

In [ ]:
z = y[::2, 5000:].mean(axis=1)
z

Use the **`compute()`** function if you want your result as a NumPy array.

In [ ]:
w = z.compute()
print(type(w), w.shape )

**Persit Data in Memory**

- If you have the available RAM for your dataset then you can persist data in memory.
- This allows future computations to be much faster.

In [ ]:
%time y.sum().compute()

In [ ]:
y = y.persist()

In [ ]:
%time y[0, 0].compute()

In [ ]:
%time y.sum().compute()

**Numpy against Dask**

In [ ]:
%%time 
x = np.random.normal(10, 0.1, size=(20000, 20000)) 
y = x.mean(axis=0)[::100] 
#y

In [ ]:
%%time
x = da.random.normal(10, 0.1, size=(20000, 20000), chunks=(1000, 1000))
x = x.persist()
y = x.mean(axis=0)[::100] 
#y.compute() 

## Dask DataFrames

- Coordinate many Pandas dataframes, partitioned along an index. 
- Support a large subset of the Pandas API.

![fig_df](https://pythondata.com/wp-content/uploads/2016/11/Screen-Shot-2016-11-24-at-6.52.24-PM-168x300.png)

The <a href="https://en.wikipedia.org/wiki/Arctic_oscillation">Arctic oscillation (AO)</a> or Northern Annular Mode/Northern Hemisphere Annular Mode (NAM) is a weather phenomenon at the Arctic poles north of 20 degrees latitude. It is an important mode of climate variability for the Northern Hemisphere.

Let us read the AO dataset using Dask.

In [ ]:
url = "http://www.cpc.ncep.noaa.gov/products/precip/CWlink/daily_ao_index/monthly.ao.index.b50.current.ascii"

In [ ]:
df = dd.read_table(url, sep='\s+', 
               parse_dates={'Dates':[0, 1]}, header=None)
df

In [ ]:
df.dtypes

In [ ]:
df.head()

**Use Standard Pandas Operations**

In [ ]:
df.compute()

In [ ]:
df.columns = ['Dates', 'AO']
df.compute()

In [ ]:
df.AO.compute()

In [ ]:
df.compute().plot(x="Dates", y="AO")

In [ ]:
df2 = df[df.AO > 0]
df2.compute().plot(x="Dates", y="AO")

In [ ]:
df3 = df[df.Dates > '1995-01-01'].AO.sum()
print(df3.compute())

## Parallelize Code with `dask.delayed`

- We want to parallelize a simple for-loop

### Simple Code

Consider the following functions:

In [ ]:
import time
import random

def increment(x):
    time.sleep(1.0)
    return x + 1

def double(x):
    time.sleep(1.0)
    return 2 * x

def add(x,y):
    time.sleep(1.0)
    return x + y

Let us use the above functions within two for-loops:

In [ ]:
%%timeit -r 1

n = 20
data = [i+1 for i in range(n)]

out = []
for x in data:
    y = increment(x)
    z = double(y)
    out.append(z)
    
total = 0
for z in out:
    total = add(total, z)

total

We can use the `dask.delayed` decorator to parallelize the functions `increment`, `double` and `add`:

In [ ]:
increment = dask.delayed(increment)
double = dask.delayed(double)
add = dask.delayed(add)

We use the `visualize` method (relies on the `graphviz` package) that provide a visual representation of the operations being performed.

In [ ]:
x = increment(1)
y = increment(2)
z = add(x, y)
z.visualize(rankdir='LR')

We can now revisit the orginal code by using the wrapped/decorated methods defined above.

In [ ]:
n = 20
data = [i+1 for i in range(n)]

out = []
for x in data:
    y = increment(x)
    z = double(y)
    out.append(z)
    
total = 0
for z in out:
    total = add(total, z)

total

- Note that we have not physically calculated **total** yet.
- We need to apply the `compute` method to get the answer.

In [ ]:
%%timeit -r 1
dask.compute(total)

We can also get the visual representation through a task graph.

In [ ]:
total.visualize()

### Schedulers

To execute the task graphs there are two types of schedulers:
* Single machine
* Distributed

**Single thread**

In [ ]:
%time total.compute(scheduler='synchronous')

**Local threads**

Uses `multiprocessing.pool.ThreadPool`

Use all the processors

In [ ]:
%time total.compute(scheduler='threads')

Use some of the processors:

In [ ]:
%time total.compute(scheduler='threads', num_workers=2)

**Local processes**

Uses `multiprocessing.Pool`

Use all the processors:

In [ ]:
import multiprocessing
print (multiprocessing.cpu_count())

In [ ]:
%time result = total.compute(scheduler='processes')

Use some of the processors:

In [ ]:
%time result = total.compute(scheduler='processes', num_workers=2)

**Run through a Dashboard**

- Requires Bokeh
- As soon as you create a client, Dask will automatically start using it.

In [ ]:
from dask.distributed import Client
client = Client()
client

In [ ]:
import random

def random_slow_add(x, y):
    time.sleep(random.randrange(8,15))
    return x + y

In [ ]:
results = []

for x in data:
    y = dask.delayed(random_slow_add)(x, 1)
    results.append(y)
    
total = dask.delayed(sum)(results)

In [ ]:
%time result = total.compute()
result

In [ ]:
# Shut down the cluster
client.close()

### Example with DataFrame

Build a Pandas DataFrame with 100K rows and two columns with values selected randomly between 1 and 1000

In [ ]:
df = pd.DataFrame({'X':np.random.randint(1000, size=100000),
                   'Y':np.random.randint(1000, size=100000)})
df

Write a function that computes the sum of square for each column of the DataFrame.

In [ ]:
def add_squares(df):
    return df.X**2+df.Y**2

Measure the time it takes to call the function:

In [ ]:
%%timeit
df['add_squares'] = df.apply(add_squares,axis=1)

In [ ]:
df

**Parallelize using Dask `Map_Partition`**

- We construct a dask dataframe from pandas dataframe using `from_pandas` function and specify the number of partitions (nparitions) to break this dataframe into.
- We will break into 4 partitions (number of available cores)

In [ ]:
ddf = dd.from_pandas(df, npartitions=4)

We will apply `add_squares` method on each of these partitions:

In [ ]:
%%timeit
ddf['z'] = ddf.map_partitions(add_squares, 
                               meta=(None, 'int64')).compute()

In [ ]:
def myfunc(x, y):
    return y * (x**2 + 1)

In [ ]:
%%timeit
df1 = df.apply(lambda row: myfunc(row.X, row.Y), axis=1)

In [ ]:
import multiprocessing
ddf = dd.from_pandas(df, npartitions=4*multiprocessing.cpu_count())
ddf

In [ ]:
%%timeit
ddfz = ddf.map_partitions(lambda data: 
                              data.apply(lambda row: myfunc(row.X, row.Y), axis=1)).compute(scheduler='processes')

# <font color="red">Dask and GPUs</font>

- Dask can also help to scale out large array and dataframe computations by combining the Dask Array and DataFrame collections with a GPU-accelerated array or dataframe library.
- The RAPIDS libraries provide a GPU accelerated Pandas-like library, `cuDF`, which interoperates well and is tested against Dask DataFrame.
- You can convert a Pandas-backed Dask DataFrame to a cuDF-backed Dask DataFrame.

MORE TO COME...